## Community Detection using transactions.


This notebook demonstrates the use of community detection algorithms to find communities using transactions.

This time, transactions table is also included in the property graph creation. 

The objective is to identify the communities where the customers in the **ring fraud** found in notebook PGX.RingsAndFraud.Community.ipynb  are, to investigate more.

In [1]:
import json
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

def generateToken(base_url, username, password):
    from urllib.request import Request, urlopen
    from urllib.error import HTTPError

    body = json.dumps({ 'username': username, 'password': password }).encode('utf8')
    headers = { 'content-type': 'application/json' }
    request = Request(base_url + '/auth/token', data=body, headers=headers)

    try:
        response = urlopen(request).read().decode('utf-8')
        return json.loads(response).get('access_token')

    except HTTPError as err:

        if err.code == 400:
            print('Authentication failed no username/password given')
        elif err.code == 401:
            print('Authentication failed invalid username/password')
        else:
            print("Server returned HTTP response code: {} for URL: {}".format(err.code, err.url))

        os._exit(1)

baseUrl = 'https://localhost:7007'
username = 'bankuser'
password = 'welcome1'

token=generateToken(baseUrl, username, password)

In [2]:
try:
    import pypgx as pgx

    session = pgx.get_session(base_url=baseUrl, token=token)
    instance = session.server_instance
    analyst = session.create_analyst()

except SystemExit:
    os._exit(0)
    

In the previous paragraph, some basic PGX objects were built:

 * *session* this is the first
 * *instance* created from the session
 * **analyst**  this one is used for the algorithms

Now read the graph from the database to the PGX engine session.

This time we'll use a different json file that includes the **PTRANSACTIONS** table.

This is the database model:

![atl text](http://localhost/bankuser_datamodel.png "bankuser schema")

Now, all is ready to load the graph from database tables to PGX memory structures.

This new graph is called **banktx**.

A json file containing the metadata and mappings from the database table is used for this step.

Let's have a look at the contents of /home/oracle/graphdata/bank_tx.json:

```json
{
    "name": "banktx",
    "vertex_id_strategy": "keys_as_ids",
    "vertex_id_type": "long",
    "vertex_providers": [
        {
            "name": "ACCOUNT",
            "format": "rdbms",
            "database_table_name": "PACCOUNTS",
            "key_column": "ID",
            "key_type": "long",
            "props": [
                {
                    "name": "ACCOUNTNUMBER",
                    "type": "string"
                },
                {
                    "name": "BALANCE",
                    "type": "long"
                }
            ]
        },
        {
            "name": "CUSTOMER",
            "format": "rdbms",
            "database_table_name": "PCUSTOMERS",
            "key_column": "ID",
            "key_type": "long",
            "props": [
                {
                    "name": "FIRSTNAME",
                    "type": "string"
                },
                {
                    "name": "LASTNAME",
                    "type": "string"
                }
            ]
        },
        {
            "name": "ADDRESS",
            "format": "rdbms",
            "database_table_name": "PADDRESSES",
            "key_column": "ID",
            "key_type": "long",
            "props": [
                {
                    "name": "STREET",
                    "type": "string"
                },
                {
                    "name": "CITY",
                    "type": "string"
                },
                {
                    "name": "STATE",
                    "type": "string"
                },
                {
                    "name": "ZIPCODE",
                    "type": "long"
                },
                {
                    "name": "LATITUDE",
                    "type": "string"
                },
                {
                    "name": "LONGITUDE",
                    "type": "string"
                }
            ]
        }
    ],
    "edge_providers": [
         {
            "name": "HAS_BANKACCOUNT",
            "format": "rdbms",
            "database_table_name": "E_HAS_ACCOUNT",
            "source_column": "FROMCUSTOMER",
            "destination_column": "TOACCOUNT",
            "source_vertex_provider": "CUSTOMER",
            "destination_vertex_provider": "ACCOUNT",
            "props": [
               {
                    "name": "CREATIONDATE",
                    "type": "timestamp"
                }
            ]
        },
        {
            "name": "HAS_ADDRESS",
            "format": "rdbms",
            "database_table_name": "E_HAS_ADDRESS",
            "source_column": "FROMCUSTOMER",
            "destination_column": "TOADDRESS",
            "source_vertex_provider": "CUSTOMER",
            "destination_vertex_provider": "ADDRESS",
            "props": []
        },
        {
            "name": "TX",
            "format": "rdbms",
            "database_table_name": "E_TRANSACTIONS",
            "source_column": "FROMACCOUNT",
            "destination_column": "TOACCOUNT",
            "source_vertex_provider": "ACCOUNT",
            "destination_vertex_provider": "ACCOUNT",
            "props": []
        }
    ]
}

```

In [3]:
# read Graph

graph = session.read_graph_with_properties("/home/oracle/graphdata/bank_tx.json")


Let's have a look at the number of nodes and edges for this graph.

It gives the number of nodes (318) and edges (873) for this graph as well as its name **banktx**.

In [4]:
print('Edges number: ' + str(graph.num_edges) + ' Nodes number: '+ str(graph.num_vertices))

Edges number: 873 Nodes number: 318


## Community Detection (Graph Algorithms)

Let's use the **analyst** object to run a couple of community detection algorithms.

**Label Propagation** is an algorithm designed to find community structures in a graph. It assigns a unique community label to each vertex in the graph, which then is updated on each iteration by looking and choosing the most frequent label amongst those from its neighbors. Convergence is achieved once the label of each vertex is the same as the most frequent one amongst its neighbors, i.e. when there are no changes in the communities assigned to the vertices in one iteration.

Usually when running an algorithm a new property is added to the vertices with the results. Then this property can be used in PGQL queries and GraphViz visualizations.

In [5]:
analyst.communities_label_propagation(graph,30)

PgxPartition(graph: banktx, components: 108)

A new property called **label_propagation** contains the identifiers of the detected communities.

In [6]:
graph.get_vertex_properties()

[VertexProperty(name: ACCOUNTNUMBER, type: string, graph: banktx),
 VertexProperty(name: BALANCE, type: long, graph: banktx),
 VertexProperty(name: CITY, type: string, graph: banktx),
 VertexProperty(name: FIRSTNAME, type: string, graph: banktx),
 VertexProperty(name: LASTNAME, type: string, graph: banktx),
 VertexProperty(name: LATITUDE, type: string, graph: banktx),
 VertexProperty(name: LONGITUDE, type: string, graph: banktx),
 VertexProperty(name: STATE, type: string, graph: banktx),
 VertexProperty(name: STREET, type: string, graph: banktx),
 VertexProperty(name: ZIPCODE, type: long, graph: banktx),
 VertexProperty(name: label_propagation, type: long, graph: banktx)]

Let's see how many communities were detected and their size.

We use PGQL for this task.

In [7]:
import pandas as pd

pgxResultSetNode = graph.query_pgql("""
SELECT e.label_propagation as comm_id, count(e) as size
from MATCH (e:ACCOUNT)
group by comm_id
order by size desc
limit 10
""")

comms = []

for i in pgxResultSetNode:
    comms.append(i)

df=pd.DataFrame(comms)
df=df.reset_index(drop=True)
df.rename(columns = {0:'COMMUNITY_ID', 1:'SIZE'}, inplace = True)
df

COMMUNITY_ID  SIZE
0             0    25
1            12     9
2             5     8
3             4     8
4             3     8
5             1     8
6             7     8
7             9     6
8            13     4
9            18     2

Let's have a look at the community the customers in the fraud ring are.

Remember their ids where: 100, 101 and 102.

In [8]:
pgxResultSetNode = graph.query_pgql("""
select id(cust), acct.label_propagation
from MATCH (cust:CUSTOMER)->(acct:ACCOUNT)
where id(cust) in (100,101,102)
""")

comms = []

for i in pgxResultSetNode:
    comms.append(i)

df=pd.DataFrame(comms)
df=df.reset_index(drop=True)
df.rename(columns = {0:'CUSTOMER_ID', 1:'COMMUNITY_ID'}, inplace = True)
df

CUSTOMER_ID  COMMUNITY_ID
0          100             0
1          101             0
2          102             0


Let's now visualize the graph using the built-in tool: GraphViz.

Open https://localhost:7007/ui and log in with the same credentials used in the top of this notebook.

Also, the session ID is needed because graphs created in this session, are only visible to the current session. Session ID can be found by inspecting the session object as shown in the paragraph below.

It's a string that looks like this: 

```
xxxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxx
```

In [9]:
print(session)

PgxSession(id: 3ce5daa5-bb0a-4937-83ff-f613c8e2c64f, name: pypgx)


Try the following query to inspect community where the ring is in. We want to know what other customers are making transactions with our suspects.

```
SELECT c,a,f,t,e
FROM MATCH (addr:ADDRESS)<-[i]-(c:CUSTOMER)-[a]->(f:ACCOUNT)-[e]->(t:ACCOUNT)
where t.label_propagation=0
``` 

Now, load the visualization settings: 

![atl text](http://localhost/gviz-settings3.png "communities")

Select the file **bankpv_settings** in the **graphdata** folder:

![atl text](http://localhost/gviz-settings2.png "communities")

You should see the following visualization. 

There are more than 100 results, then the visualization is paginated; let's change that to have all in one page.

![atl text](http://localhost/gviz-settings4.png "communities")

Open the **Settings** menu, select the **Visualization** tab and change the **Page Size** to 600. Once the **Ok** button is clicked the visualization will refresh and show all results in one page.

![atl text](http://localhost/gviz-settings5.png "communities")

![atl text](http://localhost/gviz-settings6.png "communities")


Then let's get the addresses of all these customers and put them in a map using the **geographical visualization** mode. 
Only the nodes of type **ADDRESS** have latitude and longitude information.

You should see they are located in a small region in Madrid, so maybe they are somehow contacting other customers in that area.

**NOTE**: This information was extracted from the Geolocalization done with Oracle Spatial Studio from their postal addresses. 


```
SELECT addr
FROM MATCH (addr:ADDRESS)<-[i]-(c:CUSTOMER)-[a]->(f:ACCOUNT)-[e]->(t:ACCOUNT)
where t.label_propagation=0 or f.label_propagation=0
```

Only the nodes of type ADDRESS are selected, so let's enable the Geographical visualization by clicking the **Settings**:


![gviz-settings](http://localhost/gviz-settings8.png "communities")

Then, in the **Visualization** tab, select **Geographical** and choose the correct properties for latitude and longitude as shown below:

![gviz-settings](http://localhost/gviz-settings9.png "communities")

Finally the result will look like this:

![gviz-settings](http://localhost/gviz-settings7.png "communities")


Now let's see the rest of communities, that are smaller in size and don't have links with the fraud ring.
These are spread in all the province of Madrid.


```
SELECT addr
FROM MATCH (addr:ADDRESS)<-[i]-(c:CUSTOMER)-[a]->(f:ACCOUNT)-[e]->(t:ACCOUNT)
where t.label_propagation<>0 or f.label_propagation<>0
```

![gviz-settings](http://localhost/gviz-settings10.png "communities")


This could indicate the fraud ring customers are building their network with some kind of method they have deployed in that area (finding people in the neighborhood, using some kind of local advertisement, using a company that operates in that area... ) and it's worth to investigate more and to warn other customers in that area so they don't end up in this community.

Finally destroy graph and session when finished.

In [ ]:
graph.destroy()
session.destroy()